In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

import feather

In [2]:
version = "08"

In [3]:
train = feather.read_dataframe(f'../data/processed/train_processed_{version}.feather')
test = feather.read_dataframe(f'../data/processed/test_processed_{version}.feather')

In [9]:
concat_data = pd.concat([train,test], axis=0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
y = concat_data["istrain"]
concat_data = concat_data.drop(["isFraud", "TransactionID", "istrain", "card_no"], axis=1)

In [12]:
y = y.astype(int)

In [13]:
params = {'num_leaves': 64,
         'min_data_in_leaf': 100, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.1,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 0,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 0,
         "verbosity": -1}

In [14]:
verbose=200
early_stopping_rounds=200
num_rounds=1000
sampling_strategy=0.2



In [15]:
trn_data = lgb.Dataset(
    concat_data,
    label= y
)
model = lgb.train(
    params=params,
    train_set=trn_data,
    num_boost_round=1000,
    verbose_eval=verbose,
)

In [16]:
feature_importance = pd.DataFrame()
feature_importance["feature"] = concat_data.columns
feature_importance["importance"] = model.feature_importance()

In [17]:
feature_importance.sort_values(by=["importance"], ascending=False)

,feature,importance
409,card1,1405
4,C13,1316
406,addr1,1276
469,count_enc_addr1,1266
473,count_enc_card2,1130
452,card2,1113
16,D11,1021
449,card1_TransactionAmt_std,1020
448,card1_TransactionAmt_mean,1012
414,card1_D10_std,1000
